In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Tool Creation

In [2]:
@tool
def multiply(a: int, b: int) -> int:
    """Given two numbers a and b, this tool returns their product"""
    return a * b

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
llm.invoke("hi").content

'Hi there! How can I help you today?'

## Tool Binding

In [7]:
llm_with_tools = llm.bind_tools([multiply])

llm_with_tools.invoke("Hi").content

'Hello! How can I help you today?'

## Tool Calling

In [8]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")

In [9]:
print(result)

content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--2b174304-7cd0-4389-9350-e731dad4b207-0' tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': 'cd6b437b-9518-4adb-9370-9c8777e964c1', 'type': 'tool_call'}] usage_metadata={'input_tokens': 63, 'output_tokens': 18, 'total_tokens': 158, 'input_token_details': {'cache_read': 0}}


In [10]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 2.0, 'b': 3.0},
  'id': 'cd6b437b-9518-4adb-9370-9c8777e964c1',
  'type': 'tool_call'}]

## Tool Execution

In [17]:
multiply.invoke(result.tool_calls[0]["args"])

6

In [18]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='6', name='multiply', tool_call_id='cd6b437b-9518-4adb-9370-9c8777e964c1')

In [23]:
from langchain_core.messages import HumanMessage

query = HumanMessage("What is 2 multiplied by 3?")
messages = [query]

result = llm_with_tools.invoke(messages)
messages.append(result)

messages

[HumanMessage(content='What is 2 multiplied by 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--b1de7554-d773-4221-a77f-533b4ea697d6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': '3af15874-96e4-490a-a329-5c8f459d76ad', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 18, 'total_tokens': 137, 'input_token_details': {'cache_read': 0}})]

In [24]:
tool_result = multiply.invoke(result.tool_calls[0])

messages.append(tool_result)

messages

[HumanMessage(content='What is 2 multiplied by 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--b1de7554-d773-4221-a77f-533b4ea697d6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': '3af15874-96e4-490a-a329-5c8f459d76ad', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 18, 'total_tokens': 137, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='6', name='multiply', tool_call_id='3af15874-96e4-490a-a329-5c8f459d76ad')]

In [26]:
llm_with_tools.invoke(messages).content

'2 multiplied by 3 is 6.'